In [1]:
import pandas as pd
from openai import OpenAI
from dotenv import load_dotenv
from components import Components
import os
load_dotenv()

e:\Github Repositories\RAG-FYP2024\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

#### Setting up OpenAI GPT4 Mini

In [2]:
token = os.getenv("GITHUB_TOKEN")
route = "https://models.inference.ai.azure.com"
model_name = "gpt-4o-mini"


In [3]:
client = OpenAI(
    base_url=route,
    api_key=token,
)

In [4]:
def get_response(prompt):
    response = client.chat.completions.create(
    messages=[
        {
            "role": "system",
            "content": """Instructions:
                        - Be helpful and answer questions concisely. If you don't know the answer, say 'I don't know'
                        - Be concise and to the point.""",
        },
        {
            "role": "user",
            "content": prompt,
        }
    ],
    model=model_name,
    temperature=0.1,
    max_tokens=1024,
    top_p=1.
)
    return response.choices[0].message.content

#### Setting up GroqLLM


In [2]:
rag_components = Components("Snowflake/snowflake-arctic-embed-s","mixedbread-ai/mxbai-embed-large-v1","mixtral-8x7b-32768")

In [3]:
llm = rag_components.get_groq_llm(system_prompt=""" Instructions:
        - Try your hardest to answer the question correctly.
        - Be concise and to the point.""")

#### Loading Data

In [5]:
df_qa = pd.read_csv('../data/MainDataset/data/QA.csv')

In [6]:
question_list = []
for questions in df_qa['question']:
    question_list.append(questions)

In [7]:
import time
response_list = []
for i in range(len(question_list)): 
    time.sleep(5)
    print(question_list[i])
    response = get_response(question_list[i])
    #response = llm.complete(question_list[i])
    print(response)
    rowIndex = df_qa.index[i]
    #response_list.append(response)
    df_qa.at[rowIndex, 'gen_answer'] = response
    print('')
    print("--"*100)
    print(f'{i}/{128}')
 


how many calories in fortune cookie
A typical fortune cookie contains about 30 to 50 calories.

--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
69/128
what is a signature dish
A signature dish is a meal that is strongly associated with a particular chef, restaurant, or cuisine, often showcasing unique flavors or techniques that define their culinary style.

--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
70/128
another name for joy stick
Game controller.

--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
71/128
what does arm wing liposuc

In [12]:
df.to_csv(f'../data/MainDataset/results/GPT4oMini_Run1_BaseLLMNORAG.csv',index=False)